In [1]:
import mediapipe as mp

In [2]:
wCam, hCam = 640, 480
frameR = 100
smoothening = 5

In [3]:
pTime = 0
plocX, plocY = 0, 0
clocX, clocY = 0, 0

In [4]:
import cv2

In [5]:
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

True

In [6]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(static_image_mode=False,max_num_hands=1,model_complexity=1,min_detection_confidence=0.5,min_tracking_confidence=0.5)

In [7]:
mpDraw = mp.solutions.drawing_utils
tipIds = [4, 8, 12, 16, 20]

In [8]:
import autopy

In [9]:
wScr, hScr = autopy.screen.size()
print('Screen size:', wScr, hScr)

Screen size: 1536.0 864.0


In [10]:
def findHands(img, draw=True):
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            if draw:
                mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
    return img, results

In [11]:
def findPosition(img, results, handNo=0, draw=True):
    lmList, bbox = [], []
    if results.multi_hand_landmarks:
        myHand = results.multi_hand_landmarks[handNo]
        xList, yList = [], []
        for id, lm in enumerate(myHand.landmark):
            h, w, c = img.shape
            cx, cy = int(lm.x * w), int(lm.y * h)
            xList.append(cx)
            yList.append(cy)
            lmList.append([id, cx, cy])
            if draw:
                cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
        xmin, xmax = min(xList), max(xList)
        ymin, ymax = min(yList), max(yList)
        bbox = xmin, ymin, xmax, ymax
        if draw:
            cv2.rectangle(img, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20), (0, 255, 0), 2)
    return lmList, bbox

In [12]:
def fingersUp(lmList):
    fingers = []
    if len(lmList) == 0:
        return fingers
    if lmList[tipIds[0]][1] > lmList[tipIds[0] - 1][1]:
        fingers.append(1)
    else:
        fingers.append(0)
    for id in range(1, 5):
        if lmList[tipIds[id]][2] < lmList[tipIds[id] - 2][2]:
            fingers.append(1)
        else:
            fingers.append(0)
    return fingers

In [13]:
def findDistance(p1, p2, lmList, img, draw=True, r=15, t=3):
    x1, y1 = lmList[p1][1:]
    x2, y2 = lmList[p2][1:]
    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
    if draw:
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), t)
        cv2.circle(img, (x1, y1), r, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), r, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (cx, cy), r, (0, 0, 255), cv2.FILLED)
    length = math.hypot(x2 - x1, y2 - y1)
    return length, img, [x1, y1, x2, y2, cx, cy]